# Segmenting and Clustering Neighborhoods in Toronto

### Author: Bryan Choi

Note: This notebook contains answers for all 3 parts of the assignment.

## TOC:
1. [Part 1: Data Scraping and Pre-processing](#part-1)
2. [Part 2: Importing Geospatial Data and Merging](#part-2)
3. [Part 3: Plotting Folium Map](#part-3)

## Initialisation

In [53]:
import folium
import requests
import pandas as pd
from bs4 import BeautifulSoup


## Part 1: Data Scraping and Pre-processing <a class="anchor" id="part-1"></a>

In [47]:
# Pulling data from Wikipedia table
wiki_data = requests.get(
    "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802"
).text
soup = BeautifulSoup(wiki_data, "lxml")
df = pd.read_html(str(soup.table))
df = df[0]

In [84]:
# Removing rows which Borough = "Not assigned"
df = df[df["Borough"] != "Not assigned"]

# Checking if there are other "Not assigned" cells and duplicated postal code
print("Any remaining 'Not assigned' cells >>> {} | Any duplicated postal code >>> {}".format("Not assigned" in df, True in df["Postal Code"].duplicated()))

Any remaining 'Not assigned' cells >>> False | Any duplicated postal code >>> False


In [49]:
# Previewing some data to ensure validity
print(df.head())
df.shape

  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


(103, 3)

## Part 2: Importing Geospatial Data and Merging <a class="anchor" id="part-2"></a>

In [85]:
# Importing GeoSpatial .csv
geodata = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
)

# Checking geodata df to have the same number of rows as main df
print(geodata.head())
geodata.shape

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


(103, 3)

In [51]:
# Merging main df and geodata
df1 = pd.merge(df, geodata, on="Postal Code")
df1.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3: Plotting Folium Map <a class="anchor" id="part-3"></a>

In [78]:
# Generating Folium map object
map = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# For loop to label and plot each df1 row
for lat, lng, bor, neigh in zip(
    df1["Latitude"], df1["Longitude"], df1["Borough"], df1["Neighbourhood"]
):
    popup = folium.Popup("{}, {}".format(neigh, bor), parse_html=True) # Defining popup object
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=popup,
        color="Blue",
        fill=False,
        fill_color="Blue",
        fill_opacity=1,
    ).add_to(map)

map